In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from skimage.color import gray2rgb
import shap

# Load the FashionMNIST dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Preprocess the data
X_train = X_train / 255.0
X_test = X_test / 255.0
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)


1   Loading and Preprocessing the Data: The code starts by loading the FashionMNIST dataset and normalizing the pixel values between 0 and 1

In [ ]:
# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

2   Defining and Training the Model: A convolutional neural network (CNN) model is defined using the Sequential API in Keras. The model consists of two convolutional layers followed by max-pooling, a flatten layer, and two dense layers. The model is compiled and trained using the training data.

In [ ]:

# Select two examples
example1_idx = np.argmax(y_test == 0)
example2_idx = np.argmax(y_test == 1)

example1 = X_test[example1_idx]
example2 = X_test[example2_idx]

# Convert grayscale image to RGB
example2_rgb = gray2rgb(example2.reshape(28, 28))

# Make predictions for the examples
example1_pred = model.predict(example1.reshape(1, 28, 28, 1))
example2_pred = model.predict(example2.reshape(1, 28, 28, 1))

# Get the predicted classes and probabilities
example1_class = np.argmax(example1_pred)
example1_prob = example1_pred[0, example1_class]
example1_second_prob = np.sort(example1_pred)[0, -2]
example1_least_prob = np.sort(example1_pred)[0, -1]

example2_class = np.argmax(example2_pred)
example2_prob = example2_pred[0, example2_class]
example2_second_prob = np.sort(example2_pred)[0, -2]
example2_least_prob = np.sort(example2_pred)[0, -1]


3   Selecting Examples: Two examples are selected from the test set, one from class 0 (T-shirt/top) and another from class 1 (Trouser).

4   Converting Grayscale Image to RGB: The second example is converted from grayscale to RGB format using the gray2rgb function from skimage.color. This is done to match the input format expected by the SHAP library.

5   Making Predictions: The model makes predictions for the two examples using the predict method.

6   Getting Predicted Class and Probabilities: The predicted class and probabilities for the true class, most likely class, second most likely class, and least likely class are extracted from the prediction outputs.


print(f"Example 1: True Class - {class_names[0]}")
print(f"           Predicted Class - {class_names[example1_class]}")
print(f"           Probability - {example1_prob:.4f}")
print(f"           Second Most Likely Probability - {example1_second_prob:.4f}")
print(f"           Least Likely Probability - {example1_least_prob:.4f}")
print()
print(f"Example 2: True Class - {class_names[1]}")
print(f"           Predicted Class - {class_names[example2_class]}")
print(f"           Probability - {example2_prob:.4f}")
print(f"           Second Most Likely Probability - {example2_second_prob:.4f}")
print(f"           Least Likely Probability - {example2_least_prob:.4f}")
print()

In [ ]:

# Gradient-based Explanations
example1_tensor = tf.convert_to_tensor(example1.reshape(1, 28, 28, 1))
with tf.GradientTape() as tape:
    tape.watch(example1_tensor)
    example1_pred = model(example1_tensor)

gradients = tape.gradient(example1_pred, example1_tensor)[0]

positive_gradients = np.maximum(0, gradients)
negative_gradients = np.minimum(0, gradients)

pos_importance_scores = positive_gradients / (np.max(positive_gradients) + 1e-10)
neg_importance_scores = negative_gradients / (np.abs(np.min(negative_gradients)) + 1e-10)

print("Gradient-based Explanations for Example 1:")
print("Positive Importance Scores:")
print(pos_importance_scores)
print()
print("Negative Importance Scores:")
print(neg_importance_scores)
print()


7   Gradient-based Explanations: The code calculates the importance scores using the gradients of the model's output with respect to the input for the first example. Positive and negative importance scores are obtained by taking the maximum of 0 and the gradients, and the minimum of 0 and the gradients, respectively.

8   Integrated Gradients Explanation: The code uses the SHAP library to compute the SHAP values using the Integrated Gradients method for the second example. A background dataset is randomly sampled from the training set, and a GradientExplainer object is created with the model and the background dataset. The SHAP values are then computed for the example.


In [ ]:
# Integrated Gradients Explanation
background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]

explainer = shap.GradientExplainer(model, background)
shap_values = explainer.shap_values(example2.reshape(1, 28, 28, 1))

shap.image_plot(shap_values, -example2.reshape(1, 28, 28, 1))

9   Plotting SHAP Values: The SHAP values are plotted using the shap.image_plot function to visualize the contributions of each pixel to the prediction.

The code provides local explanations for the model's predictions using gradient-based methods, including importance scores based on gradients and SHAP values based on Integrated Gradients. These explanations help in understanding the model's decision-making process and identifying the important features contributing to the predictions.